# Setting up Hyperparameters

In [1]:
epochs = 5
negative_mining = False
soft_labels = False
nb_augment = 0
load_data_on_GPU = True

In [2]:
batch_size = 3
learning_rate = 1e-4

### Selecting the correct device for training (CPU or GPU)

In [3]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


# Importing the Data

In [4]:
from dataset import POCDataReader, data_augment_

data_reader = POCDataReader(root_dir="../data/POC", load_on_gpu=False, limit=100, verbose=True)

train_data, val_data, test_data = data_reader.split([0.7, 0.1, 0.2])

data_augment_(train_data, n=nb_augment, load_on_gpu=False, verbose=True)

Loading dataset into RAM:   0%|          | 0/2744 [00:00<?, ?it/s]

	- Loading done, RAM used: 1.49GiB / free: 59.08GiB / total: 62.73GiB
	- Got a total of 100 images.


/home/piai/Documents/POC-Project/src/dataset/POC_dataset.py:36: UserWarning: Need a strictly positive integer for n for data augmentation. Will skip augmentation.
  warnings.warn("Need a strictly positive integer for n for data augmentation. Will skip augmentation.")


In [5]:
from torch.nn import Sequential
from torchvision.transforms.functional import invert
from torch.nn.functional import normalize

from pipelines import InputPipeline, SumFilters
from pipelines.filters import *


sumFilter = SumFilters(FrangiFilter(), SatoFilter())
crackBinFilter = Sequential(FrangiFilter(), CrackBinaryFilter())
bgBinFilter = Sequential(FrangiFilter(), BGBinaryFilter(60.))
skeletonFilter = SkeletonFilter(crackBinFilter)
watershedFilter = WatershedFilter(background_filter=bgBinFilter, foreground_filter=skeletonFilter)

inpip = InputPipeline(
    transformer=None,
    layer_transformer=[
#         FrangiFilter(),
#         SatoFilter(),
#         sumFilter,
#         crackBinFilter,
#         bgBinFilter,
        skeletonFilter,
#         watershedFilter,
    ]
)

inpip = inpip.to(device)

### Creating Training Dataset

In [6]:
from torch.nn.functional import normalize
from torchvision.transforms import GaussianBlur

from dataset import POCDataset

train_dataset = POCDataset(
    train_data,
    transform=inpip,
    target_transform= GaussianBlur(kernel_size=3, sigma=0.7) if soft_labels else None,
    negative_mining=negative_mining,
    load_on_gpu=load_data_on_GPU,
    verbose=True
)

# train_dataset.precompute_transform(verbose=True)

	- Loading done, GPU memory used: 0.08GiB / free: 6.99GiB / total: 7.78GiB


In [7]:
from torch.utils.data import DataLoader

if load_data_on_GPU:
    training_dataloader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        sampler=train_dataset.sampler,
        shuffle= True if train_dataset.sampler is None else None,
    )
else:
    training_dataloader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        sampler=train_dataset.sampler,
        shuffle= True if train_dataset.sampler is None else None,
        num_workers=8,
        pin_memory=True,
        pin_memory_device=device,
    )

### Creating Validation Dataset

In [ ]:
from dataset import POCDataset

val_dataset = POCDataset(val_data, transform=inpip, target_transform=None, negative_mining=False)

In [ ]:
from torch.utils.data import DataLoader

if load_data_on_GPU:
    validation_dataloader = DataLoader(
        val_dataset,
        batch_size= 2 * batch_size,
        shuffle=True,
    )
else:
    validation_dataloader = DataLoader(
        val_dataset,
        batch_size= 2 * batch_size,
        shuffle=True,
        num_workers=8,
        pin_memory=True,
        pin_memory_device=device,
    )

### Creating Evaluation Dataset

In [ ]:
from dataset import POCDataset

test_dataset = POCDataset(test_data, transform=inpip, target_transform=None, negative_mining=False)

In [ ]:
from torch.utils.data import DataLoader

if load_data_on_GPU:
    evaluation_dataloader = DataLoader(
        test_dataset,
        batch_size=1,
        shuffle=True,
    )
else:
    evaluation_dataloader = DataLoader(
        test_dataset,
        batch_size=1,
        shuffle=True,
        num_workers=8,
        pin_memory=True,
        pin_memory_device=device,
    )

### Testing the dataloader

In [ ]:
from my_utils import show_img

features, masks, files, indexes = next(iter(training_dataloader))
print(features.size(), masks.size())

show_img(features)
show_img(masks)
print(files, indexes)

### Testing Dataset for proportion between classes

In [ ]:
_,train_labels,_,_ = next(iter(training_dataloader))

nb_pixel = torch.unique(train_labels, return_counts=True)[1]
print("Proportion of class 1 in this batch: {}%".format(nb_pixel[1] * 100 / (nb_pixel[0] + nb_pixel[1])))

Result is usually around 1.1% which may be too low for cross_entropy_loss.

# Building the differents modules

## Creating the models

#### Using Unet with bilinear upsampling and cropping to generate 2 classes (background and crack).

In [8]:
from models import UNet, DeepCrack, SubUNet

# model = UNet(n_channels=1, n_classes=2, bilinear=True, crop=False).to(device)
# model = DeepCrack(n_channels=inpip.nb_channel, n_classes=2).to(device)
model = SubUNet(n_channels=inpip.nb_channel, n_classes=2, deeply_supervised=False, avg_pooling=False).to(device)

ImportError: cannot import name 'SubUNet' from 'models' (/home/piai/Documents/POC-Project/src/models/__init__.py)

In [ ]:
print("Total number of parameters: {0:,}".format(sum(p.numel() for p in model.parameters() if p.requires_grad)))
print(f"Model structure: {model}")

## Creating the loss functions

#### Cross Entropy Loss

In [ ]:
import torch
from torch.nn import CrossEntropyLoss

weight = torch.tensor([.9, .1])  # class weight : 98% for background and 2% for the cracks
loss_fn = CrossEntropyLoss(weight=weight).to(device)

#### Dice Loss

In [ ]:
from loss import DiceLoss

loss_fn = DiceLoss().to(device)

#### Focal Loss

In [ ]:
import torch
from loss import FocalLoss

weight = torch.tensor([.9, .1])  # class weight : 90% for background and 10% for the cracks
loss_fn = FocalLoss(weight=weight, gamma=2).to(device)

#### Jaccard Loss

In [ ]:
from loss import JaccardLoss

loss_fn = JaccardLoss().to(device)

#### Tversky Loss

In [ ]:
import torch
from loss import TverskyLoss

loss_fn = TverskyLoss(alpha=0.3, beta=0.7).to(device)

#### Focal Tversky Loss

In [ ]:
import torch
from loss import FocalTverskyLoss

loss_fn = FocalTverskyLoss(alpha=0.3, beta=0.7, gamma=2).to(device)

#### Combined (Focal + Tversky) Loss

In [ ]:
import torch
from loss import CombinedLoss, TverskyLoss, FocalLoss

weight = torch.tensor([.3, .7])

loss_fn = CombinedLoss(
    FocalLoss(weight=weight, gamma=2),
    TverskyLoss(alpha=0.3, beta=0.7),
    ratio=0.75).to(device)

#### Bordered (Focal + Tversky) Loss

In [ ]:
import torch
from loss import BorderedLoss, TverskyLoss, FocalLoss

weight = torch.tensor([.3, .7])

loss_fn = BorderedLoss(
    border_loss=FocalLoss(weight=weight, gamma=2),
    volume_loss=TverskyLoss(alpha=0.3, beta=0.7),
    ratio=0.5).to(device)

In [ ]:
from loss import MultiscaleLoss

loss_fn = MultiscaleLoss(loss_fn)

## Creating the Optimizer

In [ ]:
from torch.optim import Adam

optimizer = Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.99))

## Setting up the Learning Rate Scheduler

In [ ]:
from torch.optim.lr_scheduler import CosineAnnealingLR

lr_scheduler = CosineAnnealingLR(optimizer, T_max=epochs)

## Creating the Metrics

In [ ]:
from metrics import Metrics, EvaluationMetrics

loss_names = loss_fn.get_names()
pipe_names = inpip.get_names()

train_metrics = Metrics(
    buffer_size=len(training_dataloader),
    mode="Training",
    hyperparam={
        'Network': model.__class__,
        'Optimizer': optimizer.__class__,
        'Loss Function': loss_fn,
        'Input Filter': pipe_names[1],
        'Input Layer': pipe_names[2],
        'Batch Size': batch_size,
        'Learning Rate': learning_rate,
        'Negative Mining': negative_mining,
        'Smooth Labeling': soft_labels,
    },
    device=device)

val_metrics = Metrics(
    buffer_size=len(validation_dataloader),
    mode="Validation",
    hyperparam={
        'Network': model.__class__,
        'Optimizer': optimizer.__class__,
        'Loss Function': loss_fn,
        'Input Filter': pipe_names[1],
        'Input Layer': pipe_names[2],
        'Batch Size': batch_size,
        'Learning Rate': learning_rate,
        'Negative Mining': negative_mining,
        'Smooth Labeling': soft_labels,
    },
    device=device)

test_metrics = EvaluationMetrics(
    buffer_size=len(evaluation_dataloader),
    hyperparam={
        'Network': model.__class__,
        'Optimizer': optimizer.__class__,
        'Loss Function': loss_fn,
        'Input Filter': pipe_names[1],
        'Input Layer': pipe_names[2],
        'Batch Size': batch_size,
        'Learning Rate': learning_rate,
        'Negative Mining': negative_mining,
        'Smooth Labeling': soft_labels,
    },
    epochs=epochs,
    device=device)

# Training, testing and validating the Model

In [ ]:
from train_tqdm import training_loop, validation_loop, evaluation_loop

for epoch in range(1, epochs+1):
    training_loop(epoch, training_dataloader, model, loss_fn, optimizer, lr_scheduler, train_metrics, device)
    validation_loop(epoch, validation_dataloader, model, loss_fn, val_metrics, device)

evaluation_loop(evaluation_dataloader, model, test_metrics, device)

train_metrics.close_tensorboard()
val_metrics.close_tensorboard()
test_metrics.close_tensorboard()